In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from shapely.geometry import Polygon
from shapely.ops import unary_union
from PIL import Image
import cv2
from rasterio import features
import rasterio
from typing import List

In [ ]:
# ROOT_DATA_DIR = Path("/home/zakirov/data/kopter_done_22_04/")
ROOT_DATA_DIR = Path("/mnt/storage/datasets/walrus/raw/walrus200/")

In [ ]:
images = sorted((ROOT_DATA_DIR / "images").glob("*.jpg"))
images

In [ ]:
annotations = sorted((ROOT_DATA_DIR / "markup").glob("*.json"))
# remove categories json, it's not used
annotations.pop(annotations.index(ROOT_DATA_DIR / "markup"/ "categories.json"))
annotations

In [ ]:
from prepare_data import read_polys, poly2mask

In [ ]:
polygons = read_polys(annotations[5])
all_poly = []
for idx, ann in enumerate(annotations):
    all_poly.extend(read_polys(ann))

In [ ]:
from shapely.geometry import box

In [ ]:
fig = plt.hist([p.area for p in all_poly], bins=40);
plt.title("All morj area distribution")
plt.xlabel("Area in pixels^2");

In [ ]:
plt.hist([box(*p.bounds).area for p in all_poly if not p.is_empty], bins=60);
plt.title("All morj area distribution (of bbox)")
plt.xlabel("Area in pixels^2");
plt.xlim(0, 30000)

In [ ]:
img = Image.open(images[5])
img

In [ ]:
from tiles import TileInference

In [ ]:
splits = TileInference().iter_split(np.array(img), channels_last=True)

In [ ]:
next(splits);

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(next(splits)[0]);

In [ ]:
np.array(img).shape

In [ ]:
W, H = Image.open(images[5]).size

In [ ]:
plt.hist([p.area for p in polygons], bins=20);

In [ ]:
morj, morj_contact = poly2mask(polygons, (H, W))

In [ ]:
np.unique(morj_contact)

In [ ]:
plt.imshow(morj_contact)

In [ ]:
from train import MorjDataset, get_train_aug
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_dataloader = DataLoader(
    MorjDataset(Path("/home/zakirov/data/morj_data/"), get_train_aug(256)),
    batch_size=64,
    num_workers=12,
    drop_last=True,
    shuffle=True,
    pin_memory=True,
)

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
plt.imshow(batch[0][33, 2])

In [ ]:
plt.imshow(batch[1][33, 0])

In [ ]:
batch[0].shape

In [ ]:
batch[1].shape